# Convert format generated by SimulationParser, and hopefully FC readout into different plots and fits.

In [93]:
%matplotlib widget
#%matplotlib inline
#%matplotlib notebook
%matplotlib qt

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.interpolate import griddata
from matplotlib.patches import Circle
import mpl_toolkits.mplot3d.art3d as art3d
import scipy

In [94]:
#names of the files used
filesout = ['anode_entrance', 'anode_exit', 'BPM_middle', 'BTV_BGC', 'collector_electrode_entrance', 'collector_main_pot_entrance', 'FC_YAG']

In [197]:
#Lets read one
filenum = 1
###You can literally just change this number from any in [0,6] and just run the whole notebook again to change the file used. 
###For example if I want to see FC_YAG, I just change the index to 6 and run the whole notebook again to generate everything again but for this file.



orig_df = pd.read_csv(filesout[filenum] + '_cupped.csv')
orig_df

,x,y,I1,I2,I3,I4,I5,I6,I7
0,-8.0,-8.000000,0,0,0,0,0,0,0
1,-8.0,-6.545455,0,0,0,0,0,0,0
2,-8.0,-5.090909,0,0,0,0,0,0,0
3,-8.0,-3.636364,0,1,0,0,4,0,0
4,-8.0,-2.181818,0,1,0,1,2,0,2
...,...,...,...,...,...,...,...,...,...
139,8.0,2.181818,2,0,1,1,0,2,0
140,8.0,3.636364,0,0,3,0,0,1,0
141,8.0,5.090909,0,0,0,0,0,0,0
142,8.0,6.545455,0,0,0,0,0,0,0


Must be split into x, y, I for each cup. Ie now we convert to x position, y position, and a charge.

In [198]:
(xs, ys) = ([-5, 5, -10, 0, 10, -5, 5], [8.66, 8.66, 0, 0, 0, -8.66, -8.66])
(xs, ys) = (np.divide(xs, 10), np.divide(ys, 10))

finalxs = []
finalys = []
finalCurr=[]
cupnum = []

for index, row in orig_df.iterrows():
    #print(index, row)
    newxs = xs+row['x']
    newys = ys+row['y']
    for i in range(len(newxs)):
        finalxs.append(newxs[i])
        finalys.append(newys[i])
        finalCurr.append(row.iloc[i+2])
        cupnum.append(i+1)

In [199]:
df = pd.DataFrame({'x': finalxs, 'y': finalys, 'I': finalCurr, 'cupNumber':cupnum}, columns = ('x', 'y', 'I', 'cupNumber') )
df

,x,y,I,cupNumber
0,-8.5,-7.134,0.0,1
1,-7.5,-7.134,0.0,2
2,-9.0,-8.000,0.0,3
3,-8.0,-8.000,0.0,4
4,-7.0,-8.000,0.0,5
...,...,...,...,...
1003,7.0,8.000,0.0,3
1004,8.0,8.000,0.0,4
1005,9.0,8.000,0.0,5
1006,7.5,7.134,0.0,6


In [200]:
#Here we convert to absolute charge, this can be added anywhere, even on the SimulationParser file if prefered.
df['I']=df['I']*0.01
df

,x,y,I,cupNumber
0,-8.5,-7.134,0.0,1
1,-7.5,-7.134,0.0,2
2,-9.0,-8.000,0.0,3
3,-8.0,-8.000,0.0,4
4,-7.0,-8.000,0.0,5
...,...,...,...,...
1003,7.0,8.000,0.0,3
1004,8.0,8.000,0.0,4
1005,9.0,8.000,0.0,5
1006,7.5,7.134,0.0,6


In [201]:
#Lets see what the scan looked like
df.plot.scatter(x='x',y='y', s=1, figsize=(5, 5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [202]:
#Just to see range of values we have
print(df.max())
print(df.min())

x            9.000
y            8.866
I            0.080
cupNumber    7.000
dtype: float64
x           -9.000
y           -8.866
I            0.000
cupNumber    1.000
dtype: float64


In [203]:
#Plot basic 3d scatter plot of the points:
fig = plt.figure(figsize=(10, 10), dpi=80)
ax = plt.axes(projection='3d')
ax.scatter(df['x'], df['y'], df['I'], c = df['I'])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Charge')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Charge')

And comapred to the real shape?

In [204]:
#Lets read one and take a look.
pd.read_csv(filesout[filenum]+'.csv').plot.scatter(x='upmm',y='anode_entrance', s=1, figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='upmm', ylabel='anode_entrance'>

In [205]:
#Plot a trisurf of the data. This will connect points linearly. More info in the plot_trisurf function.

x = df['x']
y = df['y']
z = df['I']

fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap='plasma', linewidth=0.1)
fig.colorbar(surf, shrink=0.5, aspect=5)

# Draw a circle on the x=0 'wall'
#If you want to visually add circles to roughly by-eye guess the beam shapes this is how you can do it.
if(False):
    innerRadius = 2.5
    outerRadius = 7.5
    innerC = Circle((0, 0), innerRadius, fill=False)
    outerC = Circle((0, 0), outerRadius, fill=False)

    ax.add_patch(innerC)
    art3d.pathpatch_2d_to_3d(innerC, z=0.002, zdir="z")

    ax.add_patch(outerC)
    art3d.pathpatch_2d_to_3d(outerC, z=0.002, zdir="z")

#This line will also add the scatter points so you can see where the real data is and where the spline is. (Bit messy with 1000 points)
#surf = ax.scatter(x, y, z, cmap=cm.jet, linewidth=0.1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [206]:
#Now are going to use the grid data function to interpolate the areas around our points to make them smoother
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1, projection='3d')

X = df['x']
Y = df['y']
Z = df['I']

#Create the grid we want to interpolate over. Can make the size here as small and large as you want but compare its square to the number of points you have.
xi = np.linspace(X.min(),X.max(),100)
yi = np.linspace(Y.min(),Y.max(),100)
# VERY IMPORTANT, to tell matplotlib how is your data organized
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='cubic') #options: {‘linear’, ‘nearest’, ‘cubic’}

#Plot the contour lines, again levels can be varied freely. 
CS = plt.contour(xi,yi,zi,levels=50,linewidths=0.5,color='k')
ax = fig.add_subplot(1, 2, 2, projection='3d')

#Create a meshgrid and plot the interpreted surface 
xig, yig = np.meshgrid(xi, yi)
surf = ax.plot_surface(xig, yig, np.nan_to_num(zi), linewidth=0, cmap='plasma')
#surf = ax.plot_surface(xig, yig, zi, linewidth=0)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-206-d95d60312630>:16: UserWarning: The following kwargs were not used by contour: 'color'
  CS = plt.contour(xi,yi,zi,levels=50,linewidths=0.5,color='k')


Okay, lets try fitting some functions to this instead of just basic interpolation between points. Lets start with a Gaussian in x, y:

In [207]:
# Our function to fit is going to be a sum of two-dimensional Gaussians
def gaussian(x, y, x0, y0, xalpha, yalpha, A):
    return A * np.exp( -((x-x0)/xalpha)**2 -((y-y0)/yalpha)**2)


# This is the callable that is passed to curve_fit. M is a (2,N) array
# where N is the total number of data points in Z, which will be ravelled
# to one dimension.
def _gaussian(M, *args):
    x, y = M
    arr = np.zeros(x.shape)
    for i in range(len(args)//5):
       arr += gaussian(x, y, *args[i*5:i*5+5])
    return arr

# A list of the Gaussian parameters: x0, y0, xalpha, yalpha, A
gprms = [(0, 2, 2.5, 5.4, 1.5),
         (-1, 4, 6, 2.5, 1.8),
         (-3, -0.5, 1, 2, 4),
         (3, 0.5, 2, 1, 5)
        ]


In [208]:
X = xig
Y = yig
Z = np.nan_to_num(zi)

# Plot our function as is, so we can see what we're fitting to. 
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, cmap='plasma')
ax.set_zlim(0,np.max(Z))
plt.show()

# Initial guesses to the fit parameters.
guess_prms = [(0, 0, 1, 1, 2),
              (-1.5, 5, 5, 1, 3),
              (-4, -1, 1.5, 1.5, 6),
              (4, 1, 1.5, 1.5, 6.5)
             ]
# Flatten the initial guess parameter list.
p0 = [p for prms in guess_prms for p in prms]


# We need to ravel the meshgrids of X, Y points to a pair of 1-D arrays.
xdata = np.vstack((X.ravel(), Y.ravel()))
# Do the fit, using our custom _gaussian function which understands our
# flattened (ravelled) ordering of the data points.
popt, pcov = curve_fit(_gaussian, xdata, Z.ravel(), p0, maxfev=50000)
#popt, pcov = curve_fit(_gaussian, xdata, Z.ravel(), maxfev=50000) #It's possible to do this without giving guess params but its not recommended, even giving [0,0,0,...,0] is better.

#Create our surface using the fit function.
fit = np.zeros(Z.shape)
for i in range(len(popt)//5):
    fit += gaussian(X, Y, *popt[i*5:i*5+5])

#Print fit params, and the RMS between the z surface and our fitted surface. 
print('Fitted parameters:', popt)
rms = np.sqrt(np.mean((Z - fit)**2))
print('RMS residual =', rms)

# Plot the 3D figure of the fitted function and the residuals.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, fit, cmap=cm.jet)
cset = ax.contourf(X, Y, Z-fit, zdir='z', offset=-np.max(fit), cmap='plasma')
ax.set_zlim(-np.max(fit),np.max(fit))
plt.show()

# Plot the test data as a 2D image and the fit as overlaid contours.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(fit, origin='lower', cmap='plasma', extent=(x.min(), x.max(), y.min(), y.max()))
ax.contour(X, Y, fit, colors='w')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fitted parameters: [-6.17922017e-02  5.68663037e-04  4.24739636e+00  4.41189588e+00
 -5.02491498e+01 -6.81727640e-02  2.44416809e-03  4.55827366e+00
  4.72656486e+00  4.29988263e+01 -5.84637329e-02 -4.67228639e-04
  4.04884802e+00 -4.21142072e+00  2.30147101e+01 -7.44349544e-02
  4.20708828e-03  4.80065769e+00  4.97229441e+00 -1.57588672e+01]
RMS residual = 0.008101864377347867


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [209]:
# Plot the difference so we can see where most of the irregularities are. 
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, abs(Z-fit), cmap='plasma')
ax.set_zlim(0,np.max(Z-fit))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [210]:
# Contour the residual with the beam fit.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(fit, origin='lower', cmap=cm.jet, extent=(x.min(), x.max(), y.min(), y.max()))
ax.contour(X, Y,  Z-fit, colors='w')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
x = df['x']
y = df['y']
z = df['I']

fig = plt.figure()
plt.hexbin(x, y, C=z, gridsize=20, cmap=cm.jet, bins=None)
ax.set_xlabel('x')
ax.set_ylabel('y')
#ax.set_zlabel('z')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now same as above just with a polar function. This works fairly well...

In [212]:
num_of_args = 8

def polarf(x, y, a, b, c, d, A, B, C, lamb):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan(y/x)

    return a * np.cos(theta*A) * np.exp(-(((r-b+B*np.cos(theta))**2)/c)) * (C*(np.exp(-lamb)*(lamb**r))/scipy.special.gamma(r+1)) #Standard best fit so far - argsf = (0.007,  6.4,  7,  999, 0., 0.37,  0.289,  6.4)
    #return a * np.cos(theta*A) * (C*(np.exp(-lamb)*(lamb**r))/scipy.special.gamma(r+1)) #Only piosson - argsf = (0.008,  10,  10,  999, 0.3, -1,  2.89,  9.1)
    #return a * np.cos(theta*A) * np.exp(-(((r-b)**2)/c)) #Only Gaussian - argsf = (0.005, 10, 10, 999, 0, 0, 1, 1)
    #return a * np.cos(theta*A) * qgauss(r, 1, 1) #q-Gaussian
    
# This is the callable that is passed to curve_fit. M is a (2,N) array
# where N is the total number of data points in Z, which will be ravelled
# to one dimension.
def _fitfunc(M, *args):
    x, y = M
    arr = np.zeros(x.shape)
    for i in range(len(args)//num_of_args):
       arr += polarf(x, y, *args[i*num_of_args:i*num_of_args+num_of_args])
    return arr

#----------a------b----c-----d----A---B------C-----lamb
argsf = (0.007,  6.4,  7,  999, 0., 0.37,  0.289,  6.4)

In [213]:


# Initial guesses to the fit parameters.
guess_prms = [(0.008,  10,  10,  999, 0.3, -1,  2.89,  9.1)
             ]

# Flatten the initial guess parameter list.
p0 = [p for prms in guess_prms for p in prms]

X = xig
Y = yig
Z = np.nan_to_num(zi)

# Plot the 3D figure of the fitted function and the residuals.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, cmap='plasma')
ax.plot_surface(X, Y, polarf(X, Y, *example), cmap=cm.jet)

ax.set_zlim(0,np.max(Z))
plt.show()



# We need to ravel the meshgrids of X, Y points to a pair of 1-D arrays.
xdata = np.vstack((X.ravel(), Y.ravel()))
# Do the fit, using our custom _gaussian function which understands our
# flattened (ravelled) ordering of the data points.
popt, pcov = curve_fit(_fitfunc, xdata, Z.ravel(), p0, maxfev=100000, ftol=1e-8)
#popt, pcov = curve_fit(_gaussian, xdata, Z.ravel(), maxfev=50000)

fit = np.zeros(Z.shape)
for i in range(len(popt)//num_of_args):
    fit += polarf(X, Y, *popt[i*num_of_args:i*num_of_args+num_of_args])
print('Fitted parameters:')
print(popt)

rms = np.sqrt(np.mean((Z - fit)**2))
print('RMS residual =', rms)

# Plot the 3D figure of the fitted function and the residuals.
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, fit, cmap=cm.jet)
cset = ax.contourf(X, Y, Z-fit, zdir='z', offset=-np.max(fit), cmap='plasma')
ax.set_zlim(-np.max(fit),np.max(fit))
plt.show()

# Plot the test data as a 2D image and the fit as overlaid contours.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(fit, origin='upper', cmap='plasma', extent=(x.min(), x.max(), y.min(), y.max()))
ax.contour(X, Y, fit, colors='w')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-212-1ba289baef30>:7: RuntimeWarning: invalid value encountered in power
  return a * np.cos(theta*A) * np.exp(-(((r-b+B*np.cos(theta))**2)/c)) * (C*(np.exp(-lamb)*(lamb**r))/scipy.special.gamma(r+1)) #Standard best fit so far - argsf = (0.007,  6.4,  7,  999, 0., 0.37,  0.289,  6.4)


Fitted parameters:
[2.20850769e-02 6.43477884e+00 6.95282602e+00 9.99000000e+02
 8.39879196e-04 3.73025541e-01 7.97824005e+00 6.49764001e+00]
RMS residual = 0.008268376454674632


/cvmfs/sft.cern.ch/lcg/views/LCG_100_nxcals/x86_64-centos7-gcc9-opt/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [214]:
# Plot the difference so we can see where most of the irregularities are. 
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, (Z-fit), cmap='plasma')
ax.set_zlim(0,np.max(Z-fit))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [215]:
# Contour the residual with the beam fit.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(fit, origin='lower', cmap=cm.jet, extent=(x.min(), x.max(), y.min(), y.max()))
ax.contour(X, Y,  Z-fit, colors='w')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

L. GOLINO

lgolino@cern.ch

166064

`BELOW ARE SOME FUNCTION THAT I WAS USING TO PLAY AROUND WITH THE FITTING PROCESS, SOME OF THESE MAY BE BETTER OPTIONS IN FUTURE BUT FOR NOW EVERYTHING BELOW CAN BE IGNORED.`
# ===========


# ===========

# ===========

In [85]:
# Our function to fit is going to be a sum of two-dimensional Gaussians
def gaussian(x, y, x0, y0, xalpha, yalpha, A):
    return A * np.exp( -((x-x0)/xalpha)**2 -((y-y0)/yalpha)**2)

# A list of the Gaussian parameters: x0, y0, xalpha, yalpha, A
gprms = [(0, 2, 2.5, 5.4, 1.5),
         (-1, 4, 6, 2.5, 1.8),
         (-3, -0.5, 1, 2, 4),
         (3, 0.5, 2, 1, 5)
        ]

# This is the callable that is passed to curve_fit. M is a (2,N) array
# where N is the total number of data points in Z, which will be ravelled
# to one dimension.
def _gaussian(M, *args):
    x, y = M
    arr = np.zeros(x.shape)
    for i in range(len(args)//5):
       arr += gaussian(x, y, *args[i*5:i*5+5])
    return arr

def double_gaussian( x, c1, mu1, sigma1, c2, mu2, sigma2):
    res =   c1 * np.exp( - (x - mu1)**2.0 / (2.0 * sigma1**2.0) ) \
          + c2 * np.exp( - (x - mu2)**2.0 / (2.0 * sigma2**2.0) )
    return res

def double_gauss_z(x, y, c1x, mu1x, sigma1x, c2x, mu2x, sigma2x, c1y, mu1y, sigma1y, c2y, mu2y, sigma2y, A, B):
    resx = double_gaussian(x, c1x, mu1x, sigma1x, c2x, mu2x, sigma2x)
    resy = double_gaussian(y, c1y, mu1y, sigma1y, c2y, mu2y, sigma2y)
    return A*resx + B*resy

# This is the callable that is passed to curve_fit. M is a (2,N) array
# where N is the total number of data points in Z, which will be ravelled
# to one dimension.
def _doublegaussian(M, *args):
    x, y = M
    arr = np.zeros(x.shape)
    for i in range(len(args)//14):
       arr += double_gauss_z(x, y, *args[i*14:i*14+14])
    return arr

# Initial guesses to the fit parameters.
guess_prms = [(0, 0, 1, 1, 2),
              (-1.5, 5, 5, 1, 3),
              (-4, -1, 1.5, 1.5, 6),
              (4, 1, 1.5, 1.5, 6.5)
             ]


# Initial guesses to the fit parameters.
guess_prms_double = [(2, -10, 1, 2, 10, 1, 1, -15, 1, 1, 15, 1, 1, 1)
             ]

In [64]:
### CURVE FITTING ###
# q-Gaussian stuff
def qexp(q, x):
    vec = []
    if(q==1):
        return np.exp(x)
    for element in x:
        if(1+(1-q)*element > 0):
            vec.append( (1 + (1-q)*element) ** (1/(1-q)) )
        elif(1+(1-q)*element <= 0):
            vec.append(0 ** (1/(1-q)))
    #print(vec)
    return vec
    
def cfactor(q):
    if(q < 1):
        return ( 2 * np.sqrt(np.pi) * scipy.special.gamma(1/(1-q)) ) / ( (3-q) * np.sqrt(1-q) * scipy.special.gamma((3-q)/(2*(1-q))) )
    elif(q == 1):
        return np.sqrt(np.pi)
    elif(q < 3):
        return ( np.sqrt(np.pi) * scipy.special.gamma((3-q)/(2*(q-1))) ) / ( np.sqrt(q-1) * scipy.special.gamma(1/(q-1)) ) 
    else:
        return 0

def qgauss(x, q, beta):
    #print(cfactor(q))
    return np.array([i * np.sqrt(beta) / cfactor(q) for i in qexp(q, -1 * beta * (x ** 2))])

In [65]:
import math
import scipy


num_of_args = 8

def polarf(x, y, a, b, c, d, A, B, C, lamb):
    r = np.sqrt(x**2 + y**2)
    #print(r)
    theta = np.arctan(y/x)
    #print(theta.min())
    return a * np.cos(theta*A) * np.exp(-(((r-b+B*np.cos(theta))**2)/c)) * (C*(np.exp(-lamb)*(lamb**r))/scipy.special.gamma(r+1)) #Standard best fit so far
    #return a * np.cos(theta*A) * (C*(np.exp(-lamb)*(lamb**r))/scipy.special.gamma(r+1)) #Only piosson - argsf = (0.008,  10,  10,  999, 0.3, -1,  2.89,  9.1)
    #return a * np.cos(theta*A) * np.exp(-(((r-b)**2)/c)) #Only Gaussian - argsf = (0.005, 10, 10, 999, 0, 0, 1, 1)
    #return a * np.cos(theta*A) * qgauss(r, 1, 1) #q-Gaussian

    
#----------a----b---c----d--A--B--C--lamb
argsf = (0.005, 10, 10, 999, 0, 0, 10, 1)
argsf = (0.008,  10,  10,  999, 0.3, -1,  2.89,  9.1)

x = xi
y = yi

X, Y = np.meshgrid(x, y)
Z = polarf(X, Y, *argsf)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, cmap='plasma')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …